In [2]:
import pandas as pd
import numpy as np
import os

In [ ]:
df_residencias = pd.read_csv('../planilhas/residencias.csv')
df_residencias['Longitude'] = df_residencias['Longitude'] / 1e6
df_residencias['Latitude'] = df_residencias['Latitude'] / 1e6

In [ ]:
df_residencias.head()

In [ ]:
df_pontos_onibus = pd.read_csv('../planilhas/pontos_tratados.csv')
df_pontos_onibus.head()

# Associação entre ponto de ônibus e residência

In [ ]:
def euclidean_distance(residencia, ponto_onibus):
        return np.sqrt(np.sum(np.square(np.array(residencia) - np.array(ponto_onibus))))

In [ ]:
associacoes = []
for idx_ponto, residencia in df_residencias.iterrows():
    distancias = []
    for idx_linha, ponto_onibus in df_pontos_onibus.iterrows():
        print(idx_ponto, idx_linha)
        coord_resid = np.array([residencia['Latitude'], residencia['Longitude']])
        coord_ponto = np.array([ponto_onibus['Latitude'], ponto_onibus['Longitude']])
        # distancia_ponto_residencia = euclidean_distance(coord_resid, coord_ponto)  # Aqui estamos acessando a geometria diretamente pelo índice da coluna
        # distancias.append(distancia_ponto_residencia)
        # print(distancia)
        # if distancia <= tolerancia:
    # num_assoc = np.argmin(distancias)
    # associacoes.append({'id_residencia': idx_ponto, 'ponto_onibus_associado': num_assoc, 'distancia': np.min(distancias)})
        # associacoes.append({'ponto_id': ponto['ponto_id'], 'linha_id': linha['linha_id'], 'distancia': distancia})

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, LineString

# Exemplo de dados de pontos de ônibus (substitua pelos seus dados)
dados_pontos = {
    'ponto_id': [1, 2, 3],
    'geometry': [Point([-43.87352, -16.69642]), Point([-43.88644, -16.79298]), Point([-43.87403, -16.6892])], 
    'latitude': [-16.69642, -16.79298, -16.6892],
    'longitude': [-43.87352, -43.88644, -43.87403]
}
df_pontos = pd.DataFrame(dados_pontos)
gdf_pontos = gpd.GeoDataFrame(df_pontos, geometry=gpd.points_from_xy(df_pontos.longitude, df_pontos.latitude))

# Exemplo de dados de linhas de ônibus (substitua pelos seus dados)
dados_linhas = {
    'linha_id': ['7101 - VOLTA', '7103 - IDA', '7103 - VOLTA'],
    'distancia': ['10.2 km', '16.7 km', '18.5 km'],
    'trajeto': [
        LineString([(-43.87352, -16.69642), (-43.87342, -16.69652)]),
        LineString([(-43.88644, -16.79298), (-43.88630, -16.79300)]),
        LineString([(-43.87403, -16.68920), (-43.87448, -16.68910)])
    ]
}
df_linhas = pd.DataFrame(dados_linhas)
gdf_linhas = gpd.GeoDataFrame(df_linhas, geometry='trajeto')

# Definir uma distância de tolerância em metros para associar pontos às linhas
tolerancia = 100  # por exemplo, 100 metros

associacoes = []
for idx_ponto, ponto in gdf_pontos.iterrows():
    for idx_linha, linha in gdf_linhas.iterrows():
        distancia = ponto['geometry'].distance(linha['trajeto'])  # Aqui estamos acessando a geometria diretamente pelo índice da coluna
        if distancia <= tolerancia:
            associacoes.append({'ponto_id': ponto['ponto_id'], 'linha_id': linha['linha_id'], 'distancia': distancia})

# Converter para DataFrame
df_associacoes = pd.DataFrame(associacoes)

# Exibir o DataFrame resultante com as associações
print(df_associacoes)


In [9]:
integracao = pd.read_csv('../planilhas/Matriz_Integracao.csv')
df_int = integracao.groupby(['LINHA ORIGEM', 'SENTIDO ORIGEM']).agg({'LINHA DESTINO': 'count'})
# integracao
df_int

LINHA DESTINO
LINHA ORIGEM SENTIDO ORIGEM               
1501         IDA                        63
             VOLTA                      59
1601         IDA                        55
             VOLTA                      49
1701         IDA                        59
...                                    ...
8201         VOLTA                      50
8401         IDA                        52
             VOLTA                       9
8801         IDA                        67
             VOLTA                       9

[72 rows x 1 columns]

In [14]:
linhas_com_integracao = integracao.drop_duplicates(subset=['LINHA ORIGEM'])
linhas_com_integracao = linhas_com_integracao['LINHA ORIGEM']
linhas_com_integracao

0        1501
122      1601
226      1701
342      1702
471      2201
555      2202
669      2203
756      2601
879      2603
990      2604
1108     3301
1192     3302
1264     3601
1345     4601
1445     4603
1574     4701
1693     5101
1831     5601
1931     5702
2050     5801
2190     5802
2308     5803
2403     5901
2538     5902
2656    6201R
2772     6202
2890     6203
2980     6404
3096    6404A
3200     6604
3262     6901
3342     7101
3468     7103
3601     8201
3702     8401
3763     8801
Name: LINHA ORIGEM, dtype: object